In [1]:
import numpy as np
import networkx as nx

In [42]:
def parse_input(file):
    with open(file) as file_in:
        codes = file_in.read().splitlines()
    return codes

In [107]:
def keypad_to_graph(keypad_str):
    grid = np.array([list(row) for row in keypad_str.splitlines()])
    rows, cols = grid.shape
    G = nx.Graph()
    coords_to_nodes = {}

    for r in range(rows):
        for c in range(cols):
            coord = (r, c)
            node = grid[r, c]

            if node == 'Z':  # Ignore specific nodes
                continue

            coords_to_nodes[coord] = node.item()

            # Add edges to neighbors
            for dr, dc in dir2move:
                nr, nc = r + dr, c + dc
                if 0 <= nr < rows and 0 <= nc < cols:
                    neighbor_coord = (nr, nc)
                    neighbor_node = grid[nr, nc]
                    if neighbor_node != 'Z':  # Ignore specific neighbors
                        G.add_edge(coord, neighbor_coord)

    nodes_to_coords = {v: k for k, v in coords_to_nodes.items()}

    return G, coords_to_nodes, nodes_to_coords

In [111]:
def get_one_shortest_path(code, G, nodes2coords):
    one_shortest_path = [nodes2coords[code[0]]]
    for i in range(len(code)-1):
        one_shortest_path.extend(nx.shortest_path(G, 
                                                  source=nodes2coords[code[i]], 
                                                  target=nodes2coords[code[i+1]])[1:])
    
    return one_shortest_path

In [112]:
dir2move = {(-1, 0): '^', (1, 0): 'v', (0, -1): '<', (0, 1): '>'}
G_num_keypad, coords2num, num2coords = keypad_to_graph('789\n456\n123\nZ0A')
G_dir_keypad, coords2dir, dir2coords = keypad_to_graph('Z^A\n<v>')

In [113]:
file = 'example1.txt'
codes = parse_input(file)

for code in codes:
    code = 'A' + code
    shortest_path = get_one_shortest_path(code, G_num_keypad, num2coords)
    sequence_nums = ''.join([coords2num[(x, y)] for x, y in shortest_path])
    print(code, sequence_nums)

A029A A0258963A
A980A A3698520A
A179A A321478963A
A456A A3654563A
A379A A369878963A


In [ ]:
def min_n_press(code, G):
    one_shortest_path = []
    for i in range(len(code)-1):
        one_shortest_path.extend(nx.shortest_path(G, source=code[i], target=code[i+1])[1:])
    print(one_shortest_path)
    return len(one_shortest_path) + len(code) - 1